In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [4]:
# from google.colab import drive
# drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [3]:
# csv_file = '/content/drive/MyDrive/ColabNotebooks/IDS/small_dataset.csv'
csv_file = "./raw/raw_combined"

In [4]:
data = pd.read_csv(csv_file)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2830743 entries, 0 to 2830742
Data columns (total 79 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0    Destination Port             int64  
 1    Flow Duration                int64  
 2    Total Fwd Packets            int64  
 3    Total Backward Packets       int64  
 4   Total Length of Fwd Packets   int64  
 5    Total Length of Bwd Packets  int64  
 6    Fwd Packet Length Max        int64  
 7    Fwd Packet Length Min        int64  
 8    Fwd Packet Length Mean       float64
 9    Fwd Packet Length Std        float64
 10  Bwd Packet Length Max         int64  
 11   Bwd Packet Length Min        int64  
 12   Bwd Packet Length Mean       float64
 13   Bwd Packet Length Std        float64
 14  Flow Bytes/s                  float64
 15   Flow Packets/s               float64
 16   Flow IAT Mean                float64
 17   Flow IAT Std                 float64
 18   Flow IAT Max         

In [5]:
df = data.copy()

In [6]:
df.head(n = 10)

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,54865,3,2,0,12,0,6,6,6.0,0.00000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1,55054,109,1,1,6,6,6,6,6.0,0.00000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,55055,52,1,1,6,6,6,6,6.0,0.00000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3,46236,34,1,1,6,6,6,6,6.0,0.00000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,54863,3,2,0,12,0,6,6,6.0,0.00000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
5,54871,1022,2,0,12,0,6,6,6.0,0.00000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
6,54925,4,2,0,12,0,6,6,6.0,0.00000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
7,54925,42,1,1,6,6,6,6,6.0,0.00000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
8,9282,4,2,0,12,0,6,6,6.0,0.00000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
9,55153,4,2,0,37,0,31,6,18.5,17.67767,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN


#### Drop columns following paper `Study of Multi-Class Classification Algorithms’ Performance on Highly Imbalanced Network Intrusion Datasets`)

In [7]:
zero_variance_cols = []
for col in df.columns:
    if len(df[col].unique()) == 1:
        zero_variance_cols.append(col)
zero_variance_cols

[' Bwd PSH Flags',
 ' Bwd URG Flags',
 'Fwd Avg Bytes/Bulk',
 ' Fwd Avg Packets/Bulk',
 ' Fwd Avg Bulk Rate',
 ' Bwd Avg Bytes/Bulk',
 ' Bwd Avg Packets/Bulk',
 'Bwd Avg Bulk Rate']

In [8]:
df.drop(zero_variance_cols, axis = 1, inplace = True)

#### Check `Fwd Header Length.1` và `Fwd Header Length`

In [9]:
df[[' Fwd Header Length.1', ' Fwd Header Length']]

,Fwd Header Length.1,Fwd Header Length
0,40,40
1,20,20
2,20,20
3,20,20
4,40,40
...,...,...
2830738,80,80
2830739,40,40
2830740,64,64
2830741,120,120


In [10]:
duplicate_cols = [' Fwd Header Length.1', ' Fwd Header Length']
df.drop(duplicate_cols, axis = 1, inplace = True)

In [24]:
# df.drop(['Timestamp'], axis = 1, inplace = True)

In [11]:
df.shape

(2830743, 69)

#### Drop duplicate rows

In [12]:
df.drop_duplicates(inplace = True)

In [13]:
df.shape

(2522299, 69)

#### Replace infinity value with the maximum value of each column

In [14]:
df[df.isin([np.inf, -np.inf]).any(axis = 1)]

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
65,55043,0,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1767,60018,0,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1890,80,0,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3375,443,0,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
6796,36812,0,2,0,0,0,0,0,0.0,0.0,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2823284,61647,0,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2824709,49234,0,2,0,124,0,62,62,62.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2824846,33238,0,1,1,0,0,0,0,0.0,0.0,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2824854,59112,0,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN


In [15]:
cols = df.columns[:-1] # dont get col `Label`

for col in cols:
    max_val = df.loc[df[col] != np.inf, col].max()
    min_val = df.loc[df[col] != -np.inf, col].min()
    
    df[col].replace(np.inf, max_val, inplace = True)
    df[col].replace(-np.inf, min_val, inplace = True)

In [35]:
df[df.isin([np.inf, -np.inf]).any(axis = 1)]

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label


#### Replace `NaN` value with `-1` (follow `Study of Multi-Class Classification Algorithms’ Performance on Highly Imbalanced Network Intrusion Datasets`)

In [16]:
nan_info = df.isnull().sum()
nan_info[nan_info > 0]

Flow Bytes/s    353
dtype: int64

In [17]:
df = df.fillna(-1)

In [39]:
nan_info = df.isnull().sum()
nan_info[nan_info > 0]

Series([], dtype: int64)

#### Rename cols

In [18]:
column_names = df.columns.to_list()
to_dict = {}

for col in column_names:
    new_name = col.strip()
    new_name = new_name.replace(" ", "_")
    new_name = new_name.replace("/", "_")
    to_dict[col] = new_name.lower()

df = df.rename(columns = to_dict)

In [19]:
df.columns

Index(['destination_port', 'flow_duration', 'total_fwd_packets',
       'total_backward_packets', 'total_length_of_fwd_packets',
       'total_length_of_bwd_packets', 'fwd_packet_length_max',
       'fwd_packet_length_min', 'fwd_packet_length_mean',
       'fwd_packet_length_std', 'bwd_packet_length_max',
       'bwd_packet_length_min', 'bwd_packet_length_mean',
       'bwd_packet_length_std', 'flow_bytes_s', 'flow_packets_s',
       'flow_iat_mean', 'flow_iat_std', 'flow_iat_max', 'flow_iat_min',
       'fwd_iat_total', 'fwd_iat_mean', 'fwd_iat_std', 'fwd_iat_max',
       'fwd_iat_min', 'bwd_iat_total', 'bwd_iat_mean', 'bwd_iat_std',
       'bwd_iat_max', 'bwd_iat_min', 'fwd_psh_flags', 'fwd_urg_flags',
       'bwd_header_length', 'fwd_packets_s', 'bwd_packets_s',
       'min_packet_length', 'max_packet_length', 'packet_length_mean',
       'packet_length_std', 'packet_length_variance', 'fin_flag_count',
       'syn_flag_count', 'rst_flag_count', 'psh_flag_count', 'ack_flag_count',
  

#### Mapping labels (follows paper `Implementation of Ensemble Learning and Feature Selection for Performance Improvements in Anomaly-Based Intrusion Detection Systems`)

In [20]:
labels_mapping = {
    "BENIGN" : 0,
    "DDoS" : 1,
    "DoS Hulk" : 2,
    "DoS GoldenEye" : 2,
    "DoS slowloris" : 2,
    "DoS Slowhttptest" : 2,
    "Web Attack Brute Force" : 3,
    "Bot" : 4,
    "Web Attack XSS" : 5,
    "Web Attack Sql Injection" : 5,
    "Infiltration" : 6,
    "PortScan" : 7,
    "FTP-Patator" : 8,
    "SSH-Patator" : 8,
    "Heartbleed" : 9
}

df['label'] = df['label'].map(labels_mapping)

In [21]:
df['label'].unique()

array([0, 1, 7, 4, 6, 3, 5, 8, 2, 9], dtype=int64)

In [20]:
df

,destination_port,flow_duration,total_fwd_packets,total_backward_packets,total_length_of_fwd_packets,total_length_of_bwd_packets,fwd_packet_length_max,fwd_packet_length_min,fwd_packet_length_mean,fwd_packet_length_std,...,min_seg_size_forward,active_mean,active_std,active_max,active_min,idle_mean,idle_std,idle_max,idle_min,label
0,54865,3,2,0,12,0,6,6,6.0,0.00000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,0
1,55054,109,1,1,6,6,6,6,6.0,0.00000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,0
2,55055,52,1,1,6,6,6,6,6.0,0.00000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,0
3,46236,34,1,1,6,6,6,6,6.0,0.00000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,0
4,54863,3,2,0,12,0,6,6,6.0,0.00000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2830738,53,32215,4,2,112,152,28,28,28.0,0.00000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,0
2830739,53,324,2,2,84,362,42,42,42.0,0.00000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,0
2830740,58030,82,2,1,31,6,31,0,15.5,21.92031,...,32,0.0,0.0,0,0,0.0,0.0,0,0,0
2830741,53,1048635,6,2,192,256,32,32,32.0,0.00000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,0


In [ ]:
df.reset_index()
df.to_csv("first_preprocess.csv", index = False)